In [1]:
import pandas as pd

df = pd.read_csv('research_traffic_multiple_labels.csv')
df.shape

(60000, 12)

In [2]:
print(df.dtypes)
print(df['Label'].value_counts()[1])
print(df['Label'].value_counts()[0])
df.head()

Count of Source IP        int64
Port Count                int64
Pair Count Ratio        float64
Packet Count Diff         int64
Lookup Count Diff         int64
Protocol                  int64
Average Packet Count    float64
Average Byte Count      float64
Packet Std Dev          float64
Byte Std Dev            float64
Duration per Flow       float64
Label                     int64
dtype: object
10000
10000


,Count of Source IP,Port Count,Pair Count Ratio,Packet Count Diff,Lookup Count Diff,Protocol,Average Packet Count,Average Byte Count,Packet Std Dev,Byte Std Dev,Duration per Flow,Label
0,545,545,0.0,2,43,6,0.027523,4.788991,0.163601,28.466647,0.668020,3
1,545,545,0.0,10,0,6,0.009174,1.596330,0.095342,16.589551,0.419081,3
2,52,52,0.0,3,0,6,0.000000,0.000000,0.000000,0.000000,0.018792,3
3,545,545,0.0,13,21,6,0.023853,4.150459,0.355016,61.772825,2.946281,3
4,545,545,0.0,2,131,6,0.027523,4.788991,0.163601,28.466647,2.674461,3


In [3]:
# obj_cols=df.dtypes[df.dtypes == "object"].index.values.tolist()
# print(obj_cols)

# from sklearn.preprocessing import LabelEncoder
# #Encode labels of multiple columns at once

# df[obj_cols] = df[obj_cols].astype(str)
# df[obj_cols] = df[obj_cols].apply(LabelEncoder().fit_transform)
# #
# # Print head
# #
# print(df.dtypes)
# df.head()

In [4]:
# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
# import lightgbm as lgb

# y = df.Label
# X = df.drop(['Label'],axis=1)

# # Scale the dataset using StandardScaler
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)
# X = pd.DataFrame(X_scaled, columns=X.columns)

# params = {
#     'boosting_type': 'gbdt',
#     'objective': 'binary',
#     'metric': 'binary_logloss',
#     'num_leaves': 31,
#     'learning_rate': 0.05,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.8,
#     'bagging_freq': 5,
#     'verbose': 0
# }


# # Setting up 5-fold cross-validation
# kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
# fold = 0
# results = []

In [5]:
import lightgbm as lgb
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_score, make_scorer
from sklearn.metrics import f1_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score, KFold, cross_validate
from datetime import datetime

y = df.Label
X = df.drop(['Label'],axis=1)

# Normalize the dataset using MinMaxScaler
normalizer = MinMaxScaler()
X_normalized = normalizer.fit_transform(X)
X = pd.DataFrame(X_normalized, columns=X.columns)
print("Dataset normalized using MinMaxScaler.")

# # Scale the dataset using StandardScaler
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)
# X = pd.DataFrame(X_scaled, columns=X.columns)

X_train_cv, X_unseen_test, y_train_cv, y_unseen_test = train_test_split(X, y, test_size=0.2, random_state=42)

kf = KFold(n_splits=10, shuffle=True, random_state=42)

Dataset normalized using MinMaxScaler.


In [6]:
# Initialize lists to store metrics
train_accuracies = []
test_accuracies = []
precisions = []
recalls = []
f1_scores = []
confusion_matrices = []

# Initialize LightGBM classifier
lgb_clf = lgb.LGBMClassifier(random_state=42)

start = datetime.now()

# Perform 10-fold cross-validation
for train_index, test_index in kf.split(X_train_cv):
    X_train, X_test = X_train_cv.iloc[train_index], X_train_cv.iloc[test_index]
    y_train, y_test = y_train_cv.iloc[train_index], y_train_cv.iloc[test_index]

    # Train the LightGBM classifier
    lgb_clf.fit(X_train, y_train)

    # Make predictions on the test fold
    y_pred_test = lgb_clf.predict(X_test)
    y_pred_train = lgb_clf.predict(X_train)

    # Calculate metrics for the test fold
    train_accuracies.append(accuracy_score(y_train, y_pred_train))
    test_accuracies.append(accuracy_score(y_test, y_pred_test))
    precisions.append(precision_score(y_test, y_pred_test, average='weighted'))
    recalls.append(recall_score(y_test, y_pred_test, average='weighted'))
    f1_scores.append(f1_score(y_test, y_pred_test, average='weighted'))
    confusion_matrices.append(confusion_matrix(y_test, y_pred_test))

# Evaluate on unseen test data
y_unseen_pred = lgb_clf.predict(X_unseen_test)
unseen_accuracy = accuracy_score(y_unseen_test, y_unseen_pred)
unseen_precision = precision_score(y_unseen_test, y_unseen_pred, average='weighted')
unseen_recall = recall_score(y_unseen_test, y_unseen_pred, average='weighted')
unseen_f1 = f1_score(y_unseen_test, y_unseen_pred, average='weighted')
unseen_conf_matrix = confusion_matrix(y_unseen_test, y_unseen_pred)

end = datetime.now()
print("excution time: ", end-start)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001273 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2445
[LightGBM] [Info] Number of data points in the train set: 43200, number of used features: 11
[LightGBM] [Info] Start training from score -1.787601
[LightGBM] [Info] Start training from score -1.802371
[LightGBM] [Info] Start training from score -1.788709
[LightGBM] [Info] Start training from score -1.791482
[LightGBM] [Info] Start training from score -1.787878
[LightGBM] [Info] Start training from score -1.792593
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001148 seconds.
You can set `force_row_wise=true` to remove the overhead.
A

In [7]:
# Calculate the average confusion matrix for multi-class classification
import numpy as np

# Determine the number of classes from the confusion matrices
num_classes = confusion_matrices[0].shape[0]

# Initialize an array to store the sum of confusion matrices
confusion_matrix_sum = np.zeros((num_classes, num_classes))

# Sum up the confusion matrices from all folds
for cm in confusion_matrices:
    confusion_matrix_sum += cm

# Compute the average confusion matrix
average_conf_matrix = confusion_matrix_sum / len(confusion_matrices)

# Display the average confusion matrix
print("Average Confusion Matrix:")
max_width = max(len("{:.2f}".format(value)) for row in average_conf_matrix for value in row)

# Print each row with formatted values
for row in average_conf_matrix:
    print(" ".join(f"{value:>{max_width}.2f}" for value in row))

Average Confusion Matrix:
625.70  32.60   0.20  34.00   0.20 106.40
  5.50 779.40   0.10   0.00   0.20   9.20
  0.80   0.40 798.00   0.00   0.00   0.30
  2.50   0.00   0.00 796.90   0.00   0.20
  1.50   0.00   0.00   0.00 802.00   0.00
 17.00   4.60   0.10   0.00   0.00 782.20


In [8]:
# Print the metrics for each fold
for i in range(10):
    print(f"Fold {i+1}:")
    print(f"Training Accuracy: {train_accuracies[i]}")
    print(f"Testing Accuracy: {test_accuracies[i]}")
    print(f"Precision: {precisions[i]}")
    print(f"Recall: {recalls[i]}")
    print(f"F1-Score: {f1_scores[i]}")
    print(f"Confusion Matrix:\n{confusion_matrices[i]}\n")

print(f"Average Training Accuracy: {sum(train_accuracies) / 10}")
print(f"Average Testing Accuracy: {sum(test_accuracies) / 10}")
print(f"Average Precision: {sum(precisions) / 10}")
print(f"Average recall: {sum(recalls) / 10}")
print(f"Average F1 Score: {sum(f1_scores) / 10}")

# Print the performance on the unseen data
print("Performance on Unseen Data:")
print(f"Accuracy: {unseen_accuracy}")
print(f"Precision: {unseen_precision}")
print(f"Recall: {unseen_recall}")
print(f"F1-Score: {unseen_f1}")
print(f"Confusion Matrix:\n{unseen_conf_matrix}")

Fold 1:
Training Accuracy: 0.9687962962962963
Testing Accuracy: 0.9575
Precision: 0.9589394499436584
Recall: 0.9575
F1-Score: 0.9562175861121932
Confusion Matrix:
[[592  39   0  27   0 103]
 [  4 810   0   0   0   6]
 [  2   0 771   0   0   0]
 [  1   0   0 793   0   0]
 [  3   0   0   0 804   0]
 [ 16   3   0   0   0 826]]

Fold 2:
Training Accuracy: 0.9692361111111111
Testing Accuracy: 0.9497916666666667
Precision: 0.9526863462891919
Recall: 0.9497916666666667
F1-Score: 0.9484792097067968
Confusion Matrix:
[[634  30   0  42   1 122]
 [  7 754   1   0   0  15]
 [  0   0 811   0   0   1]
 [  0   0   0 814   0   0]
 [  2   0   0   0 783   0]
 [ 17   3   0   0   0 763]]

Fold 3:
Training Accuracy: 0.9689351851851852
Testing Accuracy: 0.9564583333333333
Precision: 0.9585160149409956
Recall: 0.9564583333333333
F1-Score: 0.955493997802997
Confusion Matrix:
[[625  28   0  29   0 108]
 [  7 830   0   0   0  10]
 [  0   1 773   0   0   0]
 [  4   0   0 820   0   1]
 [  1   0   0   0 768   0]
 

In [9]:
# Calculate Weighted Feature Importance (WFI)
feature_importances = lgb_clf.feature_importances_
wfi = feature_importances / feature_importances.sum()
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': wfi
}).sort_values(by='Importance', ascending=False)

print("\nWeighted Feature Importance (WFI):")
print(feature_importance_df)


Weighted Feature Importance (WFI):
                 Feature  Importance
10     Duration per Flow    0.201333
4      Lookup Count Diff    0.157000
3      Packet Count Diff    0.156556
6   Average Packet Count    0.094278
0     Count of Source IP    0.090833
1             Port Count    0.067722
8         Packet Std Dev    0.062444
7     Average Byte Count    0.055611
2       Pair Count Ratio    0.043222
9           Byte Std Dev    0.039667
5               Protocol    0.031333


In [10]:
# import pickle
# with open('lgb_multi.pkl', 'wb') as file:
#     pickle.dump(lgb_clf, file)